# Chapter 34: Kubernetes Admission Controllers

Kubernetes provides robust mechanisms for controlling what objects can be created and how they are configured within a cluster. While Role-Based Access Control (RBAC) determines **who** can perform operations, Admission Controllers determine **what** can be done and **how** resources should be configured. They act as gatekeepers that intercept requests to the Kubernetes API server after authentication and authorization, but before the object is persisted to etcd.

This chapter explores the two primary categories of admission controllers—Validating and Mutating—and examines production-ready implementations including Open Policy Agent (OPA) Gatekeeper. Understanding admission controllers is essential for implementing security policies, enforcing organizational standards, and maintaining cluster hygiene without modifying application code.

## 34.1 ValidatingAdmissionWebhook

ValidatingAdmissionWebhooks enforce strict policy compliance by rejecting API requests that violate defined rules. Unlike mutating controllers, they cannot modify objects; they only approve or deny them. This makes them ideal for security policies, compliance checks, and organizational constraints that must be strictly enforced.

### The Validation Flow

When a user or service submits a request to the API server:

1. **Authentication**: Is the requester who they claim to be?
2. **Authorization**: Does the requester have permission to perform this action?
3. **Mutating Admission**: Can any fields be automatically adjusted?
4. **Validating Admission**: Does this object meet policy requirements?
5. **Persistence**: Object written to etcd

If any ValidatingWebhook rejects the request, the API server returns an HTTP 400-level error with the rejection message, and the object is not created.

### Webhook Configuration Structure

A ValidatingWebhookConfiguration defines which resources are validated and where to send the validation request:

```yaml
apiVersion: admissionregistration.k8s.io/v1
kind: ValidatingWebhookConfiguration
metadata:
  name: pod-security-policy
webhooks:
  - name: validate-pod.security.example.com
    # Which API operations trigger this webhook
    rules:
      - apiGroups: [""]
        apiVersions: ["v1"]
        operations: ["CREATE", "UPDATE"]
        resources: ["pods"]
        scope: "Namespaced"
    # Client configuration for the webhook service
    clientConfig:
      service:
        namespace: security
        name: pod-validator
        path: "/validate"
        port: 443
      caBundle: ${CA_BUNDLE}  # Base64-encoded CA certificate
    # Failure handling: Fail closed (reject) or open (allow)
    failurePolicy: Fail
    # How long to wait for the webhook response (default 10s)
    timeoutSeconds: 5
    # Which namespaces to include/exclude
    namespaceSelector:
      matchLabels:
        security: enforced
    # Which objects to include/exclude based on labels
    objectSelector:
      matchExpressions:
        - key: app.kubernetes.io/managed-by
          operator: NotIn
          values: ["Helm"]
    # Side effects and admission review versions
    sideEffects: None
    admissionReviewVersions: ["v1", "v1beta1"]
```

**Key Configuration Fields:**

| Field | Description | Security Implication |
|-------|-------------|---------------------|
| `failurePolicy` | `Fail` rejects requests on webhook errors; `Ignore` allows them | Always use `Fail` for security policies to prevent bypass during outages |
| `timeoutSeconds` | Maximum wait time for webhook response | Short timeouts prevent API server overload but may cause false rejections |
| `namespaceSelector` | Limits webhook to specific namespaces | Useful for exempting system namespaces or gradual policy rollout |
| `matchConditions` | CEL expressions for fine-grained filtering (Kubernetes 1.27+) | Reduces webhook load by filtering before sending to backend |

### Building a Validating Webhook Server

The webhook server must implement the Kubernetes Admission Review protocol:

```python
# webhook_server.py - Simplified Python Flask example
from flask import Flask, request, jsonify
import base64
import json

app = Flask(__name__)

@app.route('/validate', methods=['POST'])
def validate_pod():
    admission_review = request.get_json()
    
    # Extract the Pod object from the request
    pod = admission_review['request']['object']
    pod_name = pod['metadata']['name']
    namespace = admission_review['request']['namespace']
    
    # Validation logic: Ensure privileged containers are not allowed
    for container in pod['spec']['containers']:
        security_context = container.get('securityContext', {})
        if security_context.get('privileged', False):
            return jsonify({
                "apiVersion": "admission.k8s.io/v1",
                "kind": "AdmissionReview",
                "response": {
                    "uid": admission_review['request']['uid'],
                    "allowed": False,
                    "status": {
                        "code": 403,
                        "message": f"Privileged containers are forbidden in container {container['name']}"
                    }
                }
            })
    
    # Allow the request
    return jsonify({
        "apiVersion": "admission.k8s.io/v1",
        "kind": "AdmissionReview",
        "response": {
            "uid": admission_review['request']['uid'],
            "allowed": True
        }
    })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=443, ssl_context='adhoc')
```

**Production Considerations:**
- Always use TLS certificates signed by the Kubernetes CA or a trusted CA
- Implement proper logging and metrics for audit trails
- Handle timeouts gracefully to avoid blocking API requests
- Use admission review versions consistently (prefer `v1`)

### Common Validation Scenarios

**Denying Latest Tags:**
```yaml
# Validation logic pseudo-code
if container.image.endswith(":latest"):
    return deny("Using 'latest' tag is prohibited. Use specific semantic versions.")
```

**Enforcing Resource Limits:**
```yaml
# Ensure all containers have resource limits
if not container.resources.limits:
    return deny("Resource limits must be specified")
if not container.resources.limits.memory or not container.resources.limits.cpu:
    return deny("Both memory and CPU limits are required")
```

**Restricting HostPath Volumes:**
```yaml
for volume in pod.spec.volumes:
    if volume.hostPath:
        return deny("HostPath volumes are not permitted")
```

## 34.2 MutatingAdmissionWebhook

MutatingAdmissionWebhooks intercept API requests and modify the object before it is persisted. They are powerful tools for applying defaults, injecting sidecars, and ensuring consistent configurations without requiring user intervention. Because they execute before validating webhooks, mutations can prepare objects to pass subsequent validation.

### Mutation Use Cases

**Sidecar Injection:**
Automatically inject monitoring agents, service mesh proxies, or logging containers into Pods.

**Default Resource Values:**
Apply default resource requests and limits when developers omit them.

**Label Standardization:**
Automatically add labels based on namespace, user, or other contextual information.

**Security Context Defaults:**
Apply security best practices (runAsNonRoot, readOnlyRootFilesystem) if not specified.

### MutatingWebhookConfiguration Example

```yaml
apiVersion: admissionregistration.k8s.io/v1
kind: MutatingWebhookConfiguration
metadata:
  name: pod-mutation
webhooks:
  - name: mutate-pod.security.example.com
    rules:
      - apiGroups: [""]
        apiVersions: ["v1"]
        operations: ["CREATE"]
        resources: ["pods"]
    clientConfig:
      service:
        namespace: security
        name: pod-mutator
        path: "/mutate"
      caBundle: ${CA_BUNDLE}
    failurePolicy: Fail
    # Reinvocation policy allows multiple mutating webhooks to chain
    reinvocationPolicy: IfNeeded
    sideEffects: None
    admissionReviewVersions: ["v1"]
```

**Reinvocation Policy:**
- `Never`: Webhook is called once per operation
- `IfNeeded`: Webhook may be called again if subsequent webhooks modify the object (useful for ensuring your mutations persist)

### Mutation Implementation Pattern

The webhook receives the AdmissionReview, modifies the object, and returns a patch:

```python
import jsonpatch

def mutate_pod():
    admission_review = request.get_json()
    pod = admission_review['request']['object']
    
    patches = []
    
    # Add security context if missing
    if 'securityContext' not in pod['spec']:
        patches.append({
            "op": "add",
            "path": "/spec/securityContext",
            "value": {
                "runAsNonRoot": True,
                "seccompProfile": {"type": "RuntimeDefault"}
            }
        })
    
    # Inject sidecar for monitoring
    if not any(c['name'] == 'prometheus-exporter' for c in pod['spec']['containers']):
        patches.append({
            "op": "add",
            "path": "/spec/containers/-",
            "value": {
                "name": "prometheus-exporter",
                "image": "prom/node-exporter:v1.6.0",
                "ports": [{"containerPort": 9100, "name": "metrics"}]
            }
        })
    
    # Apply patch
    patch_base64 = base64.b64encode(
        json.dumps(patches).encode()
    ).decode()
    
    return jsonify({
        "apiVersion": "admission.k8s.io/v1",
        "kind": "AdmissionReview",
        "response": {
            "uid": admission_review['request']['uid'],
            "allowed": True,
            "patchType": "JSONPatch",
            "patch": patch_base64
        }
    })
```

### Idempotency and Ordering

Mutating webhooks must be **idempotent**—applying the same mutation multiple times should not create duplicate or conflicting changes. This is critical because:

1. The API server may retry failed webhook calls
2. Multiple mutating webhooks may process the same object
3. Objects may be updated multiple times

**Best Practice:** Check for existing mutations before applying changes:

```python
# Check if mutation already applied
if pod['metadata'].get('annotations', {}).get('mutated-by') == 'my-webhook':
    return allow_without_modification()

# Apply mutation
# ...
# Add annotation to track mutation
patches.append({
    "op": "add",
    "path": "/metadata/annotations/mutated-by",
    "value": "my-webhook"
})
```

## 34.3 OPA Gatekeeper

Open Policy Agent (OPA) Gatekeeper is a customizable admission webhook that integrates OPA with Kubernetes, providing a declarative approach to policy enforcement using the Rego language. It eliminates the need to write and maintain custom webhook servers for common policy scenarios.

### Architecture Components

Gatekeeper consists of three main components:

1. **Gatekeeper Controller Manager**: Runs the validating and mutating webhooks
2. **Constraint Templates**: Define the policy structure and Rego logic
3. **Constraints**: Instantiate templates for specific enforcement scenarios

### Installation

Deploy Gatekeeper using the official manifests:

```bash
kubectl apply -f https://raw.githubusercontent.com/open-policy-agent/gatekeeper/v3.13.0/deploy/gatekeeper.yaml

# Verify installation
kubectl wait --for=condition=Ready pod -l control-plane=gatekeeper-controller-manager -n gatekeeper-system --timeout=300s
```

For production deployments, configure high availability:

```yaml
# values.yaml for Helm installation
replicas: 3
auditInterval: 60
constraintViolationsLimit: 20
auditFromCache: true
```

### Constraint Templates

ConstraintTemplates define the policy logic using Rego and the schema for constraints:

```yaml
apiVersion: templates.gatekeeper.sh/v1
kind: ConstraintTemplate
metadata:
  name: k8srequiredlabels
spec:
  crd:
    spec:
      names:
        kind: K8sRequiredLabels
      validation:
        openAPIV3Schema:
          type: object
          properties:
            labels:
              type: array
              items:
                type: string
  targets:
    - target: admission.k8s.gatekeeper.sh
      rego: |
        package k8srequiredlabels
        
        violation[{"msg": msg}] {
          provided := {label | input.review.object.metadata.labels[label]}
          required := {label | label := input.parameters.labels[_]}
          missing := required - provided
          count(missing) > 0
          msg := sprintf("Missing required labels: %v", [missing])
        }
        
        violation[{"msg": msg}] {
          label := input.parameters.labels[_]
          value := input.review.object.metadata.labels[label]
          not startswith(value, "com.company")
          msg := sprintf("Label %v must start with 'com.company'", [label])
        }
```

**Template Structure:**
- **CRD Section**: Defines the parameters users can configure (labels, namespaces, etc.)
- **Rego Section**: Contains the actual policy logic
- **Violation**: Returns messages when policies are breached

### Constraints

Constraints instantiate templates to enforce specific rules:

```yaml
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sRequiredLabels
metadata:
  name: require-team-labels
spec:
  match:
    kinds:
      - apiGroups: [""]
        kinds: ["Pod", "Service", "Deployment"]
    namespaces:
      - "production"
      - "staging"
    excludedNamespaces:
      - "kube-system"
      - "gatekeeper-system"
  parameters:
    labels:
      - "app.kubernetes.io/team"
      - "app.kubernetes.io/cost-center"
```

**Match Criteria:**
- **kinds**: Which resource types to enforce
- **namespaces**: Where to apply (empty = all)
- **excludedNamespaces**: System namespaces to skip
- **labelSelector**: Filter by resource labels
- **namespaceSelector**: Filter by namespace labels

### Built-in Constraint Library

Gatekeeper provides a library of common policies:

```bash
# Clone the policy library
git clone https://github.com/open-policy-agent/gatekeeper-library.git

# Apply common constraints
kubectl apply -f gatekeeper-library/library/general/requiredlabels/template.yaml
kubectl apply -f gatekeeper-library/library/pod-security-policy/forbidden-sysctls/template.yaml
```

**Common Policy Categories:**
- **Pod Security**: Privileged containers, host namespaces, capabilities
- **Network Policies**: Required network isolation
- **Resource Management**: Required limits, ratios
- **Storage**: Allowed storage classes, volume types
- **Images**: Allowed repositories, tag requirements

### Mutation with Gatekeeper

Gatekeeper also supports mutating policies (Assign and AssignMetadata):

```yaml
apiVersion: mutations.gatekeeper.sh/v1beta1
kind: Assign
metadata:
  name: add-security-context
spec:
  applyTo:
    - groups: [""]
      kinds: ["Pod"]
      versions: ["v1"]
  match:
    scope: Namespaced
    namespaces: ["production"]
  location: "spec.securityContext.runAsNonRoot"
  parameters:
    assign:
      value: true
```

## 34.4 Policy as Code

Policy as Code (PaC) treats compliance and security rules as version-controlled artifacts, enabling automated testing, peer review, and consistent enforcement across environments. Rego, the language used by OPA, provides a declarative approach to writing complex policies.

### Rego Language Fundamentals

Rego (pronounced "ray-go") is purpose-built for evaluating policies over structured data.

**Basic Structure:**
```rego
package example

import future.keywords.if
import future.keywords.in

# Default deny
default allow := false

# Allow if user is admin
allow if {
    input.user.role == "admin"
}

# Allow if user owns the resource
allow if {
    input.user.name == input.resource.owner
}

# Deny with message
deny[msg] {
    not input.user.email
    msg := "User email is required"
}
```

**Key Concepts:**

| Concept | Description | Example |
|---------|-------------|---------|
| **Rules** | Define logic that evaluates to true/false or values | `allow if { ... }` |
| **Variables** | Local assignments within rules | `name := input.metadata.name` |
| **Iteration** | Loop over arrays/objects | `container := input.spec.containers[_]` |
| **Sets** | Mathematical set operations | `forbidden := {"latest", "master"}` |
| **Functions** | Reusable logic blocks | `is_valid_tag(tag)` |

### Writing Effective Policies

**Context-Aware Validation:**
```rego
violation[{"msg": msg}] {
    # Extract container image
    container := input.review.object.spec.containers[_]
    image := container.image
    
    # Parse image components
    contains(image, "@sha256:")  # Check for digest reference
    
    # Extract tag
    not startswith(image, "gcr.io/company-internal/")
    
    msg := sprintf("Container %v uses external image repository", [container.name])
}
```

**Testing Policies:**
Rego includes a testing framework:

```rego
package k8srequiredlabels

test_allowed {
    result := violation with input as {
        "review": {
            "object": {
                "metadata": {
                    "labels": {
                        "app.kubernetes.io/team": "platform"
                    }
                }
            }
        },
        "parameters": {
            "labels": ["app.kubernetes.io/team"]
        }
    }
    count(result) == 0
}

test_missing_label {
    result := violation with input as {
        "review": {
            "object": {
                "metadata": {
                    "labels": {}
                }
            }
        },
        "parameters": {
            "labels": ["app.kubernetes.io/team"]
        }
    }
    count(result) == 1
}
```

Run tests with OPA CLI:
```bash
opa test . --verbose
```

### Policy Testing in CI/CD

Integrate policy testing into your CI pipeline:

```yaml
# .github/workflows/policy-test.yaml
name: Policy Tests
on: [push, pull_request]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Setup OPA
        run: |
          curl -L -o opa https://openpolicyagent.org/downloads/v0.58.0/opa_linux_amd64_static
          chmod +x opa
          sudo mv opa /usr/local/bin/
      
      - name: Test Rego Policies
        run: |
          opa test policies/ --verbose --coverage
          
      - name: Validate Constraints
        run: |
          # Dry-run constraints against live cluster
          kubectl apply --dry-run=server -f constraints/
```

## 34.5 Image Policy Admission

Container images are a critical attack vector. Image Policy Admission controllers ensure only trusted, scanned, and compliant images run in your cluster.

### ImagePolicyWebhook (Deprecated)

The legacy `ImagePolicyWebhook` requires a separate external service:

```yaml
# /etc/kubernetes/manifests/kube-apiserver.yaml
apiVersion: v1
kind: Pod
metadata:
  name: kube-apiserver
spec:
  containers:
  - name: kube-apiserver
    image: k8s.gcr.io/kube-apiserver:v1.28.0
    command:
    - kube-apiserver
    - --enable-admission-plugins=ImagePolicyWebhook
    - --admission-control-config-file=/etc/kubernetes/admission-control.yaml
```

**Limitations:**
- Requires API server configuration changes
- Single point of failure if webhook unavailable
- Limited to binary allow/deny decisions

### Modern Approach: OPA Gatekeeper Image Policies

More flexible image validation using Gatekeeper:

```yaml
apiVersion: templates.gatekeeper.sh/v1
kind: ConstraintTemplate
metadata:
  name: k8sallowedrepos
spec:
  crd:
    spec:
      names:
        kind: K8sAllowedRepos
      validation:
        openAPIV3Schema:
          properties:
            repos:
              type: array
              items:
                type: string
  targets:
    - target: admission.k8s.gatekeeper.sh
      rego: |
        package k8sallowedrepos
        
        violation[{"msg": msg}] {
          container := input.review.object.spec.containers[_]
          satisfied := [good | repo = input.parameters.repos[_] ; good = startswith(container.image, repo)]
          not any(satisfied)
          msg := sprintf("Container %s image %s not from allowed repo", [container.name, container.image])
        }
        
        violation[{"msg": msg}] {
          container := input.review.object.spec.initContainers[_]
          satisfied := [good | repo = input.parameters.repos[_] ; good = startswith(container.image, repo)]
          not any(satisfied)
          msg := sprintf("InitContainer %s image %s not from allowed repo", [container.name, container.image])
        }
```

**Usage:**
```yaml
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sAllowedRepos
metadata:
  name: allowed-repos
spec:
  match:
    kinds:
      - apiGroups: [""]
        kinds: ["Pod"]
  parameters:
    repos:
      - "gcr.io/company-project/"
      - "docker.io/library/"
      - "registry.company.internal/"
```

### Image Digest Verification

Enforce immutable image references:

```rego
violation[{"msg": msg}] {
    container := input.review.object.spec.containers[_]
    image := container.image
    
    # Require digest reference (@sha256:...)
    not contains(image, "@sha256:")
    
    msg := sprintf("Container %s must use image digest, not tag", [container.name])
}
```

### Integration with Image Scanners

Combine admission control with vulnerability scanning:

```yaml
# Workflow:
# 1. CI builds image
# 2. Image scanned by Trivy/Clair
# 3. If clean, signed with Cosign
# 4. Admission controller verifies signature

apiVersion: templates.gatekeeper.sh/v1
kind: ConstraintTemplate
metadata:
  name: k8simagesigned
spec:
  targets:
    - target: admission.k8s.gatekeeper.sh
      rego: |
        package k8simagesigned
        
        # Check for Cosign signature annotation
        violation[{"msg": msg}] {
          container := input.review.object.spec.containers[_]
          not data.external.cosign.verified[container.image]
          msg := sprintf("Container %s image %s not signed", [container.name, container.image])
        }
```

## 34.6 Resource Validation

Ensuring workloads specify appropriate resource requests and limits prevents cluster instability and noisy neighbor problems.

### ResourceQuota Enforcement

While ResourceQuotas limit aggregate namespace consumption, admission controllers can enforce granular resource specifications:

```yaml
apiVersion: templates.gatekeeper.sh/v1
kind: ConstraintTemplate
metadata:
  name: k8srequiredresources
spec:
  crd:
    spec:
      names:
        kind: K8sRequiredResources
      validation:
        openAPIV3Schema:
          properties:
            limits:
              type: array
              items:
                type: string
  targets:
    - target: admission.k8s.gatekeeper.sh
      rego: |
        package k8srequiredresources
        
        violation[{"msg": msg}] {
          container := input.review.object.spec.containers[_]
          not container.resources.limits
          msg := sprintf("Container %s must have resource limits", [container.name])
        }
        
        violation[{"msg": msg}] {
          required := {limit | limit := input.parameters.limits[_]}
          container := input.review.object.spec.containers[_]
          missing := required - object.keys(container.resources.limits)
          count(missing) > 0
          msg := sprintf("Container %s missing required limits: %v", [container.name, missing])
        }
```

**Constraint:**
```yaml
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sRequiredResources
metadata:
  name: require-limits
spec:
  match:
    kinds:
      - apiGroups: [""]
        kinds: ["Pod"]
  parameters:
    limits:
      - "cpu"
      - "memory"
```

### Limit Range Validation

Ensure containers don't request excessive resources:

```rego
violation[{"msg": msg}] {
    container := input.review.object.spec.containers[_]
    limit := container.resources.limits.memory
    # Parse memory string (e.g., "64Gi")
    limit_bytes := units.parse_bytes(limit)
    limit_bytes > 64 * 1000 * 1000 * 1000  # 64GB
    
    msg := sprintf("Container %s memory limit %s exceeds maximum 64Gi", [container.name, limit])
}
```

### Ratio Enforcement

Ensure request-to-limit ratios for predictable scheduling:

```rego
violation[{"msg": msg}] {
    container := input.review.object.spec.containers[_]
    req := to_number(container.resources.requests.cpu)
    lim := to_number(container.resources.limits.cpu)
    
    # Ensure request is at least 25% of limit
    req < lim * 0.25
    
    msg := sprintf("Container %s request must be at least 25%% of limit", [container.name])
}
```

## 34.7 Custom Admission Controllers

For specialized requirements, you may need to build custom admission controllers using Kubernetes libraries.

### Controller-Runtime Framework

The `controller-runtime` library simplifies webhook development:

```go
// main.go
package main

import (
    "context"
    "net/http"
    "os"
    
    corev1 "k8s.io/api/core/v1"
    "sigs.k8s.io/controller-runtime/pkg/client"
    "sigs.k8s.io/controller-runtime/pkg/client/config"
    "sigs.k8s.io/controller-runtime/pkg/log/zap"
    "sigs.k8s.io/controller-runtime/pkg/manager"
    "sigs.k8s.io/controller-runtime/pkg/manager/signals"
    "sigs.k8s.io/controller-runtime/pkg/webhook"
)

func main() {
    log := zap.New()
    
    mgr, err := manager.New(config.GetConfigOrDie(), manager.Options{
        Port: 9443,
        CertDir: "/certs",
    })
    if err != nil {
        log.Error(err, "unable to create manager")
        os.Exit(1)
    }
    
    // Setup webhook
    mgr.GetWebhookServer().Register("/validate-pod", &webhook.Admission{
        Handler: &PodValidator{Client: mgr.GetClient()},
    })
    
    log.Info("starting manager")
    if err := mgr.Start(signals.SetupSignalHandler()); err != nil {
        log.Error(err, "unable to run manager")
        os.Exit(1)
    }
}

// PodValidator implements admission.Handler
type PodValidator struct {
    client.Client
}

func (v *PodValidator) Handle(ctx context.Context, req admission.Request) admission.Response {
    pod := &corev1.Pod{}
    if err := v.Decoder.Decode(req, pod); err != nil {
        return admission.Errored(http.StatusBadRequest, err)
    }
    
    // Validation logic
    if pod.Spec.SecurityContext == nil || pod.Spec.SecurityContext.RunAsNonRoot == nil {
        return admission.Denied("Pod must have runAsNonRoot: true")
    }
    
    return admission.Allowed("")
}
```

### Deployment Pattern

Deploy custom webhooks with proper certificate management:

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: custom-webhook
  namespace: security
spec:
  replicas: 2
  selector:
    matchLabels:
      app: custom-webhook
  template:
    metadata:
      labels:
        app: custom-webhook
    spec:
      serviceAccountName: webhook-sa
      containers:
      - name: webhook
        image: company/custom-webhook:v1.0.0
        ports:
        - containerPort: 9443
          name: webhook
        volumeMounts:
        - name: certs
          mountPath: /certs
          readOnly: true
        resources:
          requests:
            memory: "128Mi"
            cpu: "100m"
          limits:
            memory: "256Mi"
            cpu: "500m"
        livenessProbe:
          httpGet:
            path: /healthz
            port: 8080
          initialDelaySeconds: 5
        readinessProbe:
          httpGet:
            path: /readyz
            port: 8080
          initialDelaySeconds: 5
      volumes:
      - name: certs
        secret:
          secretName: webhook-server-cert
---
apiVersion: v1
kind: Service
metadata:
  name: custom-webhook
  namespace: security
spec:
  selector:
    app: custom-webhook
  ports:
  - port: 443
    targetPort: 9443
```

### Certificate Management

Use cert-manager to automate TLS certificate rotation:

```yaml
apiVersion: cert-manager.io/v1
kind: Certificate
metadata:
  name: webhook-server-cert
  namespace: security
spec:
  dnsNames:
    - custom-webhook.security.svc
    - custom-webhook.security.svc.cluster.local
  issuerRef:
    kind: ClusterIssuer
    name: selfsigned-issuer
  secretName: webhook-server-cert
```

---

## Best Practices and Troubleshooting

### Security Best Practices

1. **Defense in Depth**: Use admission controllers as the last line of defense, not the only one. Combine with RBAC, Pod Security Standards, and network policies.

2. **Fail Closed**: Always set `failurePolicy: Fail` for security-critical webhooks to prevent policy bypass during outages.

3. **Scope Restrictions**: Use `namespaceSelector` and `objectSelector` to avoid processing system components or already-compliant workloads.

4. **Certificate Rotation**: Automate certificate management with cert-manager to prevent expiration-related outages.

5. **Audit Logging**: Enable audit logging for admission decisions:
   ```yaml
   apiVersion: audit.k8s.io/v1
   kind: Policy
   rules:
   - level: RequestResponse
     resources:
     - group: ""
       resources: ["pods"]
   ```

### Performance Optimization

**Webhook Latency:**
- Keep webhook response times under 1 second
- Use in-memory caching for reference data
- Implement circuit breakers for external dependencies

**Availability:**
- Run webhooks with multiple replicas
- Use PodDisruptionBudgets to ensure availability during updates
- Consider the impact of cluster upgrades on webhook availability

### Troubleshooting

**Webhook Not Triggering:**
```bash
# Check webhook configuration
kubectl get validatingwebhookconfiguration
kubectl get mutatingwebhookconfiguration

# Verify service endpoints
kubectl get endpoints -n security

# Check webhook logs
kubectl logs -n security -l app=gatekeeper-controller-manager
```

**Certificate Errors:**
```bash
# Verify CA bundle matches service certificate
kubectl get validatingwebhookconfiguration <name> -o yaml | grep caBundle

# Check certificate expiration
openssl x509 -in <cert-file> -text -noout | grep "Not After"
```

**Policy Conflicts:**
When multiple webhooks conflict:
1. Check execution order (mutating before validating)
2. Verify reinvocation policies
3. Review API server logs for detailed error messages

---

## Chapter Summary and Preview

This chapter explored Kubernetes Admission Controllers as the policy enforcement layer of the API server. We distinguished between **ValidatingAdmissionWebhooks**, which reject non-compliant requests, and **MutatingAdmissionWebhooks**, which modify resources to enforce defaults and inject configurations. Through practical examples, we demonstrated how to build webhook servers that enforce security policies such as prohibiting privileged containers and ensuring resource limits are defined.

We examined **OPA Gatekeeper** as a production-ready framework for Policy as Code, eliminating the need to maintain custom webhook infrastructure while providing a rich library of pre-built policies. The Rego language enables sophisticated policy logic including context-aware validation and set operations. We covered image admission strategies ensuring only trusted, scanned containers from approved registries execute in the cluster, and resource validation patterns that prevent resource starvation and noisy neighbor scenarios.

Finally, we explored building **Custom Admission Controllers** using the controller-runtime framework for specialized organizational requirements, including proper certificate management and high-availability deployment patterns.

**Key Takeaways:**
- Always configure webhooks with `failurePolicy: Fail` for security-critical policies to prevent bypass during outages.
- Prefer established frameworks like Gatekeeper over custom webhooks unless you require specific integration unavailable in existing solutions.
- Ensure mutating webhooks are idempotent to handle retries and multiple mutations gracefully.
- Test admission policies thoroughly in CI/CD using OPA's testing framework before deployment.
- Monitor webhook latency and availability as they sit in the critical path of API requests.

**Next Chapter Preview:**
Chapter 35: GitOps with ArgoCD introduces a declarative approach to continuous delivery that leverages Git as the single source of truth for infrastructure and application configurations. We will explore how ArgoCD automates the deployment of Kubernetes resources by synchronizing cluster state with Git repositories, implementing the principles of GitOps including automated synchronization, drift detection, and self-healing. The chapter covers ArgoCD architecture, application management, multi-cluster deployments, and integration with admission controllers to create a complete, auditable deployment pipeline that complements the policy enforcement mechanisms established in this chapter.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='33. advanced_helm.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='35. gitops_with_argocd.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
